# Finding scope 3 emissions of a Pharma company.
- Scope 3 category is downstream transportation and distribution
- Data taken from data.world
- Distance-based methology will be used for calculations
    - According to GHG "If sub-contractor fuel data cannot be easily obtained in order to use the fuel-based method, then the distance-based method should be used."

# Data points required for calculations of scope 3
1. Total distance in KM 
    - Coordinates of places
2. Size of packages
3. Mode of Transport
4. Emission Factors
5. Calculation formulaes
    1. Emissions from road transport: = ∑ (mass of goods purchased (tonnes) × distance travelled in transport leg × emission factor of transport mode or vehicle type (kg CO2e/tonne-km)
    2. emissions from air transport: = ∑ (quantity of goods purchased (tonnes) x distance travelled in transport leg x emission factor of transport mode or vehicle type (kg CO2e/tonne-km))
    3. emissions from sea transport: = ∑ (quantity of goods purchased (tonnes) x distance travelled in transport leg x emission factor of transport mode or vehicle type (kg CO2e/tonne-km))

### Steps I took
1. Searching for dataset
2. Understanding the dataset and making calculative assumptions
    - Assumptions made should have proper justification.
3. Listing down the data points required for calculations
4. Listing down the calculation formulaes
5. Filtering out the data columns required from main data
6. Exploratory Data Analysis
7. Choosing a single year for calculations


## Data columns
1. Country - Destination location
2. Mode - Mode of transport (Air, Ocean, Road)
3. Weight - Kilogram of weight
4. Delivery Date - Date of delivery
5. Manufacturing Site - Source location

In [21]:
# Import libraries
import pandas as pd
import numpy as np

# For geolocation
from geopy.geocoders import ArcGIS

# For calculation of source airport/port
from math import radians, sin, cos, sqrt, atan2

# Calculating the time
import time

# For converting country name to iso_2 codes
import country_converter as coco

In [2]:
# Collecting data
raw_df = pd.read_csv("./Data/SCMS_Delivery_History_Dataset_20150929.csv")

In [3]:
# Filtering out required columns from the main data.

filtered_df = raw_df[['ID','Country','Shipment Mode','Manufacturing Site','Weight (Kilograms)','Delivery Recorded Date','Item Description']].copy()
filtered_df.rename(columns={"Shipment Mode":"Mode","Weight (Kilograms)":"Weight","Delivery Recorded Date":"Delivery_Date",'Manufacturing Site':'Manufacturing_site'},inplace=True)

In [4]:
# Converting Delivery Date to datetime datatype
filtered_df.loc[:,'Delivery_Date'] = pd.to_datetime(filtered_df['Delivery_Date'])

# Exploratory Data Analysis

In [5]:
# Checking total number of data points for each year
filtered_df['Delivery_Date'].groupby(by=filtered_df['Delivery_Date'].dt.year).size()

Delivery_Date
2006      65
2007     670
2008    1109
2009    1192
2010    1176
2011    1049
2012    1250
2013    1205
2014    1599
2015    1009
Name: Delivery_Date, dtype: int64

In [6]:
# Check for null values
filtered_df.isna().sum()

ID                      0
Country                 0
Mode                  360
Manufacturing_site      0
Weight                  0
Delivery_Date           0
Item Description        0
dtype: int64

Null values found in the mode of transport. 

In [7]:
# Checking Mode column's null values

# filtered_df[filtered_df['Mode'].isna()]

# Counting the number of null values per year
filtered_df[filtered_df['Mode'].isna()].groupby(by=filtered_df['Delivery_Date'].dt.year).size()

Delivery_Date
2006      2
2007    264
2008     94
dtype: int64

Since mode of transport is not available for these years (2006,2007,2008) I will not consider these years for our base year.
Even though year 2006 has only 2 NaN rows for Mode, the number of data points are only 65. Thus not enough to consider as base year as compared to other years.

In [8]:
# Checking Weight column values
filtered_df[filtered_df['Weight'].str.isnumeric() == False]

,ID,Country,Mode,Manufacturing_site,Weight,Delivery_Date,Item Description
8,46,Nigeria,Air,"Aurobindo Unit III, India",See ASN-93 (ID#:1281),2006-12-07,"Stavudine 30mg, capsules, 60 Caps"
12,62,Nigeria,Air,"EY Laboratories, USA",Weight Captured Separately,2007-01-10,"HIV 1/2, InstantChek HIV 1+2 Kit, 100 Tests"
15,68,Zimbabwe,Air,"BMS Meymac, France",Weight Captured Separately,2007-03-19,"#102198**Didanosine 200mg [Videx], tablets, 60..."
16,69,Nigeria,NaN,ABBVIE GmbH & Co.KG Wiesbaden,Weight Captured Separately,2007-05-07,"HIV 1/2, Determine Complete HIV Kit, 100 Tests"
31,262,South Africa,NaN,GSK Mississauga (Canada),Weight Captured Separately,2008-01-29,"Zidovudine 10mg/ml [Retrovir], oral solution, ..."
...,...,...,...,...,...,...,...
10318,86817,Zimbabwe,Truck,"Cipla, Goa, India",See DN-4307 (ID#:83920),2015-07-20,"Lamivudine/Nevirapine/Zidovudine 30/50/60mg, d..."
10319,86818,Zimbabwe,Truck,"Mylan, H-12 & H-13, India",See DN-4307 (ID#:83920),2015-07-20,"Lamivudine/Nevirapine/Zidovudine 30/50/60mg, d..."
10320,86819,C�te d'Ivoire,Truck,Hetero Unit III Hyderabad IN,See DN-4313 (ID#:83921),2015-08-07,"Lamivudine/Zidovudine 150/300mg, tablets, 60 Tabs"
10321,86821,Zambia,Truck,Cipla Ltd A-42 MIDC Mahar. IN,Weight Captured Separately,2015-09-03,Efavirenz/Lamivudine/Tenofovir Disoproxil Fuma...


As you can see above the weight column has non-numeric values 'Weight Captured Separately', thus the values for these rows are not available. While values like 'See ASN-93 (ID#:1281)' can be found by mapping to that particular ID and getting the weight

In [9]:
# Counting the number of 'Weight Captured Separately' values in weight columns per year.

filtered_df[(filtered_df['Weight'].str.isnumeric() == False) & (filtered_df['Weight'] == 'Weight Captured Separately')].groupby(by=filtered_df['Delivery_Date'].dt.year).size()

Delivery_Date
2006      6
2007     28
2008    233
2009    262
2010     67
2011     56
2012     33
2013     80
2014    386
2015    356
dtype: int64

Seeing that 2012 has least number of 'Weight Captured Separately' values and it has good number of data points i.e. 1250. We can consider it as our base year.

# Data Cleaning

## Fixing Weight column

In [10]:
# Filtering out 'Weight Captured Separately' rows from Final dataset
final_df = filtered_df[(filtered_df['Delivery_Date'].dt.year == 2012) & (filtered_df['Weight'] != 'Weight Captured Separately')].copy()
final_df.reset_index(inplace = True, drop = True)

In [11]:
# Function to resolve "See DN-2947 (ID#:83642)" given in weight column and adding the weight

def mapWeights(weight):
    try:
        if weight.isnumeric() == False:
            ID = weight[-6:-1]
            weight_returned = filtered_df[filtered_df['ID'] == int(ID)]['Weight'].iloc[0]
            if weight_returned == 'Weight Captured Separately':
                return None
#             print(f'{ID} -- {weight_returned}')
            return float(weight_returned)
        return float(weight)
    except Exception as e:
        print(f'Error == {e} \n {weight[-6:-1]} --- {filtered_df[filtered_df["ID"] == int(weight[-6:-1])]["Weight"].iloc[0]}')

In [12]:
weights = final_df['Weight'].apply(mapWeights)

In [13]:
# Adding weights list to the df
final_df.loc[:,'Weight'] = weights

In [14]:
# Removing None value rows for weight

final_df = final_df[final_df['Weight'].isna() == False]

In [15]:
# Converting Weights from KG to Tonnes
# 1 kilogram = 0.001 tonne

final_df['Weight'] = final_df['Weight'] * 0.001

## Getting Location coordinates of Source Location and Destination Location

### If Mode of Transport is,
1. Air and Truck, then we get the Airport coordinates of the Capital City of the Country given
2. Ocean, then we get the coordinates of the main port of the country.

## To-Do
- Source 
1. Get the appropriate location name, port/airport.
2. Get the coordinates
- Destination
1. Get the coordinates

In [16]:
# Getting airports data
raw_airport_db = pd.read_csv('Data/airports.csv')

In [17]:
# Getting required columns 
db = raw_airport_db[['type','latitude_deg','longitude_deg','name','iso_country','municipality']]
final_airport_db = db[db['type'].str.lower().str.contains('airport')]

In [18]:
final_airport_db

,type,latitude_deg,longitude_deg,name,iso_country,municipality
1,small_airport,38.704022,-101.473911,Aero B Ranch Airport,US,Leoti
2,small_airport,59.947733,-151.692524,Lowell Field,US,Anchor Point
3,small_airport,34.864799,-86.770302,Epps Airpark,US,Harvest
5,small_airport,34.942803,-97.818019,Fulton Airport,US,Alex
6,small_airport,34.305599,-112.165001,Cordes Airport,US,Cordes
...,...,...,...,...,...,...
74638,medium_airport,40.542524,122.358600,Yingkou Lanqi Airport,CN,"Laobian, Yingkou"
74639,medium_airport,41.784401,123.496002,Shenyang Dongta Airport,CN,"Dadong, Shenyang"
74641,small_airport,-11.584278,47.296389,Glorioso Islands Airstrip,TF,Grande Glorieuse
74642,small_airport,32.110587,-97.356312,Fainting Goat Airport,US,Blum


In [83]:
# Initialising geolocation object
nom = ArcGIS()
# Initialising Country converted object
cc = coco.CountryConverter()
# Lists to append Source Latitude and Longitudes
source_latitude = []
source_longitude = []
data = {'site':[],'latitude':[],'longitude':[],'mode':[]}
source_locations = pd.DataFrame(data)

In [90]:
source_locations[source_locations['site'] == 'Mylan (formerly Matrix) Nashik']

,site,latitude,longitude,mode
10,Mylan (formerly Matrix) Nashik,19.8841,73.840567,air
17,Mylan (formerly Matrix) Nashik,19.8841,73.840567,air
20,Mylan (formerly Matrix) Nashik,19.8841,73.840567,air
21,Mylan (formerly Matrix) Nashik,19.8841,73.840567,air
22,Mylan (formerly Matrix) Nashik,19.8841,73.840567,air
23,Mylan (formerly Matrix) Nashik,19.8841,73.840567,air
24,Mylan (formerly Matrix) Nashik,19.8841,73.840567,air
25,Mylan (formerly Matrix) Nashik,NaN,NaN,ocean
28,Mylan (formerly Matrix) Nashik,NaN,NaN,ocean
29,Mylan (formerly Matrix) Nashik,19.8841,73.840567,air


# Code optimize
- Check out this website for faster haversine code
https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas
https://stackoverflow.com/questions/37324332/how-to-find-the-nearest-neighbors-for-latitude-and-longitude-point-on-python

In [19]:
# Functions to get the distance in KM
def distance(lat1, lon1, lat2, lon2):
    # Calculate the distance between two coordinates using the haversine formula
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [101]:
# Function to get the coordinates of the location
# Now that we have location of the manufacturing site coordinates. We need to get the Airport/Port location from which the item was transported
def getSourceLocation(data):
    # Code to Measure time taken by program to execute.

#   print(data)
    place = data['Manufacturing_site']
    mode = data['Mode'].lower()
    try:
        """
        Check in source location DF if the place and mode are already calculated.
        If size == 0, it means that for this place and mode the source has not been calculated.
        If already calculated, then skip to next manufacturing site
        """
        # TO:DO: ERROR in if condition
        if source_locations.size == 0 or source_locations[source_locations['site'].str.contains(place) & source_locations['mode'].str.contains(mode)].size == 0:
            location = nom.geocode(place)
            country = nom.reverse(str(location.latitude) + ',' + str(location.longitude)).address.split(',')[-1].replace(" ", "")
            # Getting iso_2 code for the country
            iso2_country = coco.convert(names=country, to='ISO2')

            # Now that we have location of the manufacturing site. We need to get the Airport/Port location from which the item was transported
            if mode == 'air':
                # Finding the airport which has minimum distance
                min_distance = float('inf')
                min_latitude = float('inf')
                min_longitude = float('inf')

                # Getting all airports lying inside the country
                airports = final_airport_db[final_airport_db['iso_country'] == iso2_country]

                for index,row in airports.iterrows():
                    dist = distance(location.latitude, location.longitude, row['latitude_deg'],row['longitude_deg'])
                    if dist <= min_distance:
                        min_distance = dist
                        min_latitude = row['latitude_deg']
                        min_longitude = row['longitude_deg']

                source_latitude.append(min_latitude)
                source_longitude.append(min_longitude)
                source_locations.loc[len(source_locations.index)] = [place,min_latitude,min_longitude,mode]
            else:
                source_locations.loc[len(source_locations.index)] = [place,None,None,mode]
                source_latitude.append(None)
                source_longitude.append(None)
    except Exception as e:
        print(e, place)
        source_latitude.append(None)
        source_longitude.append(None)
        source_locations.loc[len(source_locations.index)] = [place,None,None,mode]

In [86]:
# Adding source_latitude and source_longitude columns
# store starting time
begin = time.time()
coordinates = final_df.apply(getSourceLocation,axis=1)
end = time.time()
# total time taken
print(f"Total runtime of the program is {end - begin}")

Total runtime of the program is 1.2902247905731201
Total runtime of the program is 2.486147880554199
Total runtime of the program is 1.8911259174346924
Total runtime of the program is 1.8790922164916992
Total runtime of the program is 0.5897426605224609
Total runtime of the program is 1.2590670585632324
Total runtime of the program is 0.0010001659393310547
Total runtime of the program is 0.6249198913574219
Total runtime of the program is 2.0071115493774414
Total runtime of the program is 0.0010004043579101562
Total runtime of the program is 0.0010001659393310547
Total runtime of the program is 0.0010001659393310547
Total runtime of the program is 1.8461894989013672
Total runtime of the program is 0.0009989738464355469
Total runtime of the program is 0.0
Total runtime of the program is 0.7019712924957275


D:\Anaconda\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Total runtime of the program is 0.9726741313934326
Total runtime of the program is 1.3570027351379395
Total runtime of the program is 0.9712409973144531
Total runtime of the program is 0.0019197463989257812


2643 not found in ISOnumeric


Total runtime of the program is 0.6609077453613281
Total runtime of the program is 1.3711163997650146
Total runtime of the program is 0.0
Total runtime of the program is 0.0009999275207519531
Total runtime of the program is 0.0009756088256835938
Total runtime of the program is 1.1753578186035156
Total runtime of the program is 1.1046721935272217
Total runtime of the program is 0.0010030269622802734
Total runtime of the program is 0.0


...

D:\Anaconda\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Total runtime of the program is 0.5606653690338135
Total runtime of the program is 0.0010004043579101562


D:\Anaconda\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Total runtime of the program is 1.008225679397583
Total runtime of the program is 0.0009658336639404297
Total runtime of the program is 0.000997781753540039
Total runtime of the program is 0.0010001659393310547
Total runtime of the program is 0.0
Total runtime of the program is 0.0010285377502441406
Total runtime of the program is 0.0010013580322265625
Total runtime of the program is 0.0009970664978027344
Total runtime of the program is 0.0009942054748535156
Total runtime of the program is 0.0010066032409667969


KeyboardInterrupt: 

In [55]:
# Combining source longitude and latitude coordinates to main dataframe
final_df['Source_latitude'] = source_latitude
final_df['Source_longitude'] = source_longitude

ValueError: Length of values (640) does not match length of index (1202)

In [ ]:
# Some source coordinates were not found, thus removing those rows.
## Alternatively, I can rerun the API on None values just to confirm whether the data is not available.

final_df = final_df[final_df['Source_latitude'].isna() == False]

In [ ]:
final_df